In [ ]:
pip install transformers torchvision pillow

In [ ]:
# https://cdn.dailyvet.co.kr/wp-content/uploads/2024/05/15231647/20240515ceva_experts4.jpg
from transformers import BlipProcessor, BlipForQuestionAnswering
from PIL import Image
import requests

# 1. URL에서 이미지 로드
image_url = "https://cdn.dailyvet.co.kr/wp-content/uploads/2024/05/15231647/20240515ceva_experts4.jpg"  # 이미지 URL
response = requests.get(image_url, stream=True)
image = Image.open(response.raw).convert("RGB")

# 2. 모델과 프로세서 불러오기
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

# 3. 질문 입력
question = "What is the color of the cat?"

# 4. 입력 데이터 처리
inputs = processor(image, question, return_tensors="pt")

# 5. 모델 예측
output = model.generate(**inputs)

# 6. 답변 디코딩
answer = processor.decode(output[0], skip_special_tokens=True)
print(f"Answer: {answer}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Answer: gray and white


In [ ]:
# ml-1m 데이터 다운로드 및 압축 해제
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -o ml-1m.zip


--2024-11-21 04:39:55--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  3.54MB/s    in 1.6s    

2024-11-21 04:39:57 (3.54 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [ ]:
# 파일 확인
!ls ml-1m/

movies.dat  ratings.dat  README  users.dat


In [21]:
# 모든 장르 추출
unique_genres = set("|".join(movies['genres']).split("|"))

# 결과 출력
print("Unique genres in the dataset:")
print(unique_genres)


Unique genres in the dataset:
{'Horror', 'Documentary', 'Musical', 'Thriller', 'Drama', 'Crime', 'Action', 'Sci-Fi', 'Mystery', 'Romance', 'War', 'Western', 'Comedy', 'Animation', 'Film-Noir', "Children's", 'Adventure', 'Fantasy'}


In [25]:
from transformers import BlipProcessor, BlipForQuestionAnswering
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

# 모델과 프로세서 불러오기
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

### 이미지 캡셔닝 모델, 프로세서 불러오기
processor_cap = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model_cap = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [3]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

movies = pd.read_csv(
    '/content/drive/My Drive/ml-1m/movies.dat',
    sep='::',
    engine='python',
    header=None,
    names=['movie_id', 'title', 'genres'],
    encoding='latin-1'
)
# 2. 텍스트 데이터 생성 (제목 + 장르)
movies['text'] = movies['title'] + " | " + movies['genres']

# 확인
print(movies.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   movie_id                               title                        genres  \
0         1                    Toy Story (1995)   Animation|Children's|Comedy   
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy   
2         3             Grumpier Old Men (1995)                Comedy|Romance   
3         4            Waiting to Exhale (1995)                  Comedy|Drama   
4         5  Father of the Bride Part II (1995)                        Comedy   

                                             text  
0  Toy Story (1995) | Animation|Children's|Comedy  
1   Jumanji (1995) | Adventure|Children's|Fantasy  
2        Grumpier Old Men (1995) | Comedy|Romance  
3         Waiting to Exhale (1995) | Comedy|Drama  
4     Father of the Bride Part II (1995) | Comedy  


In [100]:
# 영화 포스터 이미지 열기
# image_path = "/content/drive/My Drive/toystory.png"
# image_path = "/content/drive/MyDrive/intotheworld.webp"
# image_path = "/content/drive/MyDrive/Sabrina.jpg"
image_path = "/content/drive/MyDrive/베테랑2.jpeg"
# image_path = "/content/drive/MyDrive/30일.webp"
# image_path = "/content/drive/MyDrive/파일럿.jpeg"
image = Image.open(image_path)

In [101]:
# 질문 생성
question = "What movie is this?"  # 영화 제목을 묻는 질문
# 또는
# question = "What genre is this movie?"  # 영화 장르를 묻는 질문

In [102]:
# 입력 처리
inputs = processor(images=image, text=question, return_tensors="pt")

# 답변 생성
out = model.generate(**inputs)
answer = processor.decode(out[0], skip_special_tokens=True)

print(f"Answer: {answer}")

### 이미지 캡셔닝 처리
inputs = processor_cap(images=image, return_tensors="pt")
outputs = model_cap.generate(**inputs)
caption = processor_cap.decode(outputs[0], skip_special_tokens=True)
print("Caption:", caption)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Answer: van heusen
Caption: the movie poster for the movie ' s upcoming release


In [103]:
# 모델이 반환한 답을 기반으로 유사한 영화 추천 (영화 제목이 일치하는 영화 찾기)
recommended_movies = movies[movies['title'].str.contains(answer, case=False)]

print(f"Recommended Movies: {recommended_movies.head()}")


Recommended Movies: Empty DataFrame
Columns: [movie_id, title, genres, text]
Index: []


In [104]:
from difflib import get_close_matches
# 영화 장르를 묻는 질문
#- 구체적인 질문(예시를 구체적으로 제공) -> 비교적 정확한 답변 나옴 (코사인 유사도 65%)
question = f"Given the movie '{caption}', its main themes and plot, can you classify it into one of these genres? Action, Drama, Comedy, Sci-Fi, Fantasy, Horror, Animation, Thriller, or Romance?"

#- 추상적인 질문 -> 베테랑 포스터로 했을때 전혀 다른 결과가 나옴 (코사인 유사도 0%)
# question = f"Given that '{caption}', what genre would it best fit?"

# question = f"Given the movie title '{caption}', can you tell me its primary genre?"


# 답변 생성
inputs = processor(images=image, text=question, return_tensors="pt")
out = model.generate(**inputs)
answer = processor.decode(out[0], skip_special_tokens=True)

print(f"Genre: {answer}")

available_genres = {
    'Horror', 'Documentary', 'Musical', 'Thriller', 'Drama', 'Crime', 'Action',
    'Sci-Fi', 'Mystery', 'Romance', 'War', 'Western', 'Comedy', 'Animation',
    'Film-Noir', "Children's", 'Adventure', 'Fantasy'
}



# 1. 입력된 장르를 genres와 유사하게 매핑하는 함수 추가
def match_genre(predicted_genre, available_genres):
    # 간단히 단어가 포함되는지 확인
    for genre in available_genres:
        if predicted_genre.lower() in genre.lower():
            return genre

    return None

# 2. 장르 후보 리스트 생성
available_genres = set("|".join(movies['genres']).split("|"))

# 3. 모델이 반환한 장르를 매칭
# answer = "romance"
def find_closest_genre(predicted_genre, available_genres):
    closest_matches = get_close_matches(predicted_genre, available_genres, n=1, cutoff=0.1)
    return closest_matches[0] if closest_matches else None

matched_genre = find_closest_genre(answer, available_genres)

# matched_genre = match_genre(answer, available_genres)

# 4. 매칭된 장르로 영화 추천
if matched_genre:
    recommended_movies = movies[movies['genres'].str.contains(matched_genre, case=False)]
    print(f"Matched Genre: {matched_genre}")
    print(f"Recommended Movies: {recommended_movies.head()}")
else:
    print("No matching genre found.")

# 해당 장르에 맞는 영화 추천
# recommended_movies = movies[movies['genres'].str.contains(answer, case=False)]
# print("결과\n")
# print(f"Recommended Movies: {recommended_movies.head()}")


Genre: action movie
Matched Genre: Action
Recommended Movies:     movie_id                    title                     genres  \
5          6              Heat (1995)      Action|Crime|Thriller   
8          9      Sudden Death (1995)                     Action   
9         10         GoldenEye (1995)  Action|Adventure|Thriller   
14        15  Cutthroat Island (1995)   Action|Adventure|Romance   
19        20       Money Train (1995)                     Action   

                                                 text  
5                 Heat (1995) | Action|Crime|Thriller  
8                        Sudden Death (1995) | Action  
9        GoldenEye (1995) | Action|Adventure|Thriller  
14  Cutthroat Island (1995) | Action|Adventure|Rom...  
19                        Money Train (1995) | Action  


In [105]:

# 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

available_genres = {
    'Horror', 'Documentary', 'Musical', 'Thriller', 'Drama', 'Crime', 'Action',
    'Sci-Fi', 'Mystery', 'Romance', 'War', 'Western', 'Comedy', 'Animation',
    'Film-Noir', "Children's", 'Adventure', 'Fantasy'
}

predicted_genre = answer

# 영화 장르 리스트로 벡터화 준비
genre_list = list(available_genres)

# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(genre_list + [predicted_genre])

# 유사도 계산
similarity_scores = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])

# 유사도 출력
print(f"Similarity Scores: {similarity_scores.flatten()}")
print(answer)

Similarity Scores: [0.         0.         0.         0.         0.65945268 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]
action movie


In [106]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

### 매칭된 장르와 인풋 영화와의 연관성 코사인 유사도

# matched_genre와 answer 두 텍스트 간의 유사도 계산
texts_to_compare = [matched_genre, answer]

# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts_to_compare)

# 코사인 유사도 계산
similarity_score = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

# 결과 출력
print(f"Similarity between matched_genre and answer: {similarity_score[0][0]}")


Similarity between matched_genre and answer: 0.5797386715376658
